In [90]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pprint

In [91]:
from selenium.webdriver.edge.options import Options

# Specify your Edge profile path
profile_path = r"/Users/Ayush.Chakraborty/Library/Application Support/Microsoft Edge/Default"

options = Options()
options.add_argument(f"user-data-dir={profile_path}")  # Add the profile path

service = EdgeService(executable_path=r"../edgedriver_mac64/msedgedriver")
browser = webdriver.Edge(service=service)



In [92]:
browser.get("https://sellercenter.daraz.com.np/apps/seller/login")

In [93]:

phone_input = browser.find_element(By.XPATH, '//*[@id="account"]')
phone_input.send_keys('9811400555')

password_input = browser.find_element(By.XPATH, '//*[@id="password"]')
password_input.send_keys('Alam@5555')

login_button = browser.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/div/div[2]/div/div/div[2]/form/div[3]/button')
login_button.click()

In [94]:
browser.get("https://sellercenter.daraz.com.np/apps/product/publish")

In [95]:
import json

# Path to your JSON file
file_path = 'categories_reference.json'  # Adjust the path if necessary

# Load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)   

paths = []

def find_paths(d, current_path=None):
    if current_path is None:
        current_path = []
    
    for k, v in d.items():
        new_path = current_path + [k]  # Create a new path as a list
        if isinstance(v, dict) and v:  # Check if the value is a non-empty dict
            find_paths(v, new_path)
        else:  # Leaf node (empty dict)
            paths.append(new_path)  

In [96]:
find_paths(data)

In [97]:
path_str=[' > '.join(path) for path in paths]

In [ ]:
path_str

In [99]:
def ul_elements(browser):
    try:
        ul_list=browser.find_element(By.CSS_SELECTOR, ".next-overlay-wrapper.opened").text.split('\n')
        return ul_list
    except:
        return []
            
            

In [100]:
def required_attribute(string):
    # Split the input string into lines
    lines = string.split('\n')

    # Extract elements that have '*' before them
    extracted_elements = [lines[i + 1].strip() for i in range(len(lines) - 1) if lines[i].strip() == '*']
    return extracted_elements

In [103]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time
attribute_cache={}

def attribute_extractor_obj(browser,path):
    wait = WebDriverWait(browser, 10)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/section/div[2]/div/div[1]/div/div/div[2]/form/div/div[1]/div[2]/div/div/div/div[2]/div[1]/div[2]/button')))

    # Scroll the element into view and allow some time for the scrolling
    browser.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(2)

    # Try clicking with exception handling for intercepted clicks
    try:
        element.click()
        time.sleep(2)  # Extra wait after the click
    except ElementClickInterceptedException:
        print("Element click intercepted, retrying with JavaScript click.")
        # Retry the click using JavaScript if intercepted
        browser.execute_script("arguments[0].click();", element)
        time.sleep(2)
     # Allow time for the UI to update

    # Locate the grid containing attribute options
    grid = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/section/div[2]/div/div[1]/div/div/div[2]/form/div/div[1]/div[2]/div/div/div/div[2]/div[1]/div[1]/div")))

    all_fields=grid.text
    text = required_attribute(all_fields) or ['']
    attribute_obj = []
    cache_key=path.split(' > ')[0]+' '+','.join(text)
    if len(text)<=1:
         return [{
                "name": "Brand",
                "type": "list",
                "multiselect": False,
                "value": []
            }]
    elif cache_key in attribute_cache:
        return attribute_cache[cache_key]
    else: 
        
        input = grid.find_elements(By.TAG_NAME, 'input')
        attribute_obj.append({
                "name": "Brand",
                "type": "list",
                "multiselect": False,
                "value": []
            })
        for i in range(1,len(text)):
                # Click the input field
            browser.execute_script("arguments[0].scrollIntoView(true);", input[i])  # Ensure element is in view
            input[i].click()
            time.sleep(3)  # Give time for dropdowns to appear

            # Determine the type of input field
            type_obj = 'list'
            value = ul_elements(browser)

            if 'Not Found' in value:
                value = []
            elif len(value) == 0:
                value = None
                type_obj = "text"

            # Check if the input field allows multiple selection
            multiselectable = 'aria-multiselectable="true"' in input[i].get_attribute('outerHTML')

            # Append attribute details to the list
            attribute_obj.append({
                "name": text[i],
                "type": type_obj,
                "multiselect": multiselectable,
                "value": value
            })
        attribute_cache[cache_key]=attribute_obj
                

        return attribute_obj

In [104]:
attributes = []

In [ ]:
wait = WebDriverWait(browser, 10)
batch_start=3000
browser.refresh()
for path in path_str[batch_start:4000]:
    print(f'Processing {path_str.index(path)}: {path}')
    if "test" not in path.lower():
        try:
            # Wait for the category dropdown to be clickable
            category_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Please select category or search with keyword']")))
            category_dropdown.click()
            time.sleep(3)


            # Click on the search field to search categories
            search_category = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@placeholder="Search category"]')))
            search_category.click()
            time.sleep(3)
            search_category.send_keys(path)
            time.sleep(2)

            # Select the first category from the dropdown list
            first_cat_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.next-overlay-wrapper.opened > div > div > div.gc-category-cascade-view.aplus-module-auto-exp > div > div.category-search-input.aplus-auto-exp > div ul li:first-child")))
            first_cat_select.click()
            time.sleep(3)

            # Confirm category selection
            confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-spm='d_category_selector_confirm_btn']")))
            confirm_button.click()
            time.sleep(5)
    

            # Extract attributes
            attribute = attribute_extractor_obj(browser,path)
            attributes.append({
                'Category': path,
                'Attribute': attribute
            })

            # Refresh the page
            browser.refresh()
            time.sleep(5)

        except Exception as e:
            # Handle errors and continue with the next category
            print(f"Error processing category: {path}")
            attributes.append({
                'Category': path,
                'Attribute': None
            })
            browser.refresh()
            time.sleep(5)
            continue

In [112]:
failed_items=[[attributes.index(i), i['Category']] for i in attributes if i['Attribute']==None]

In [113]:
len(failed_items)

0

In [ ]:
for path in failed_items:
    print(f'Processing {path_str.index(path[1])}: {path[1]}')
    if "Test" not in path[1]:
        try:
            # Wait for the category dropdown to be clickable
            category_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Please select category or search with keyword']")))
            category_dropdown.click()
            time.sleep(3)

            # Click on the search field to search categories
            search_category = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@placeholder="Search category"]')))
            search_category.click()
            time.sleep(2)
            search_category.send_keys(path[1])

            # Select the first category from the dropdown list
            first_cat_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.next-overlay-wrapper.opened > div > div > div.gc-category-cascade-view.aplus-module-auto-exp > div > div.category-search-input.aplus-auto-exp > div ul li:first-child")))
            first_cat_select.click()
            time.sleep(3)

            # Confirm category selection
            confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-spm='d_category_selector_confirm_btn']")))
            confirm_button.click()
            time.sleep(5)

            # Extract attributes
            attribute = attribute_extractor_obj(browser, path[1])

            attributes[path[0]]['Attribute']=attribute
            # attributes.append({
            #     'Category': path,
            #     'Attribute': attribute
            # })
            attributes

            # Refresh the page
            browser.refresh()
            time.sleep(5)

        except Exception as e:
            # Handle errors and continue with the next category
            print(f"Error processing category: {path[1]}, Error:{e}")
            # attributes.append({
            #     'Category': path,
            #     'Attribute': None
            # })
            browser.refresh()
            time.sleep(5)
            continue


In [114]:
import json
with open('Attributes_batch5.json', 'w') as json_file:
    json.dump(attributes, json_file, indent=4)

In [125]:
import json
with open('total_attributes.json', 'w') as json_file:
    json.dump(path_str, json_file, indent=4)